# Auto Script for running at night or unattended

Change the 'hours' variable depending on how many datapoints you want to collect.Keep in mind that if you want to collect 5 hours of data at 1hr intervals, hours=10 because for example at hour4 you get a new post and you want 5 datapoints (num_data=5) then you have to run the script for 5 more hours. (The code won't collect new posts after hour5). If you want to collect data every 30min over a 5hr interval you have to manually calculate num_data which is basically how many times 30min goes into 5hr (num_data=10, hours=10, time2sleep=1800).

Right now the script is set to collect data every 30min interval

Change the 'sub_title' variable to whatever subreddit name is.

'time2sleep' variable is how many seconds you want the program to go to sleep before continuing data collection. ie if we want data collected every hour then time2sleep=3600 seconds. You still have to keep the program and your computer on and running during the sleeping phase though.

In [ ]:
import time
import praw
import pandas as pd

reddit = praw.Reddit(client_id='eOqj6Q2BiQQyKQ', client_secret='1hqeAwOvlkY1PYw51ewhbrItd5k', user_agent='Reddit WebScrapping')

sub_title = 'funny' #title of subreddit to scrape

time2sleep = 1800 #how long the program should sleep before continuing
hours = 48 #how long(hours) you want to run the script for
num_data = 48 #how many datapoints we want per post
count = 0 #each count represents 1 interval passing

votes = {} #store post ids and votes. Positive votes and NEGATIVE votes (when post.downs is > 0)
totalcomments = {} #store post ids and the corresponding number of comments
timestamps = {} #store timestamp of post creation just so we can tell what day/hour
online = {'timestamp':[],'users_online':[]} #store number of online users and timestamp of subreddit

subred = reddit.subreddit(sub_title)

while (count < hours):
    
    print('Starting hour {}'.format(count))
    
    if count < int(hours/2):
        sub = reddit.subreddit(sub_title).new(limit=20) #collect first X values in 'new' section
    else:
        pass
        
    online['timestamp'].append(time.time())
    online['users_online'].append(subred.active_user_count)
    
    #first loop through should produce nothing since there is nothing inside post_id
    for id_val in votes.keys():
        '''
        We want to follow each post for 24hrs so at max we need 24 datapoints per post.
        A fresh post collected at hour 23 would then need an extra 24hrs to get 24 datapoints
        but we also do not want to collect more data from the previous posts at hour1,2, etc.
        '''
        if len(votes[id_val]) != num_data: #if we do not have X datapoints
            result = reddit.submission(id=id_val) #go to the specific post
            votes[id_val].append(result.ups - result.downs) #get the current number of upvotes and add to the list
            totalcomments[id_val].append(result.num_comments) #get number of comments
        else:
            pass #do nothing if we already collected data from a post for 24hrs
        
    for post in sub:
        pid = str(post.id)
        if pid not in votes: #if new post, add it to the dictionary
            votes[pid] = [post.ups - post.downs] #get the upvotes of the post
            totalcomments[pid] = [post.num_comments] #get number of comments
            timestamps[pid] = [post.created_utc] #store the timestamps of each post. still need to convert later.
        else:#just in case we get some repeating posts
            pass
    if count == hours - 1:
        print('finished collecting data')
        break
    else:
        print('sleeping...')
        time.sleep(time2sleep)
        count+=0.5 #change count depending to +=0.5 for 30min intervals or +=1 for 1hr intervals
    
upvotes = pd.DataFrame(list(votes.values()),index=votes.keys())
upvotes.to_csv('./votes.csv',index_label='id')

comments = pd.DataFrame(list(totalcomments.values()),index=totalcomments.keys())
comments.to_csv('./comments.csv',index_label='id')

times = pd.DataFrame(list(timestamps.values()),index=timestamps.keys())
times.to_csv('./post_time.csv',index_label='id')

on = pd.DataFrame(list(online.values()),index=online.keys())
on.to_csv('./online_users.csv',index_label='timestamp')

# Manual Script

Run the first 2 cells below when you first begin in order to initialize stuff. Afterwards, run the 3rd cell (the one that only has one line) every 1hr or 30mins or however often you want to collect data.

In [32]:
#run this cell once for initialization or if you made changes to the function

import time
import praw
import pandas as pd

def get_data(sub_title, subred, reddit, votes, totalcomments, timestamps, online, num_data):
    
    sub = reddit.subreddit(sub_title).new(limit=20) #collect first X values in 'new' section
    online['timestamp'].append(time.time())
    online['users_online'].append(subred.active_user_count)
    
    #first loop through should produce nothing since there is nothing inside post_id
    for id_val in votes.keys():
        '''
        We want to follow each post for 24hrs so at max we need 24 datapoints per post.
        A fresh post collected at hour 23 would then need an extra 24hrs to get 24 datapoints
        but we also do not want to collect more data from the previous posts at hour1,2, etc.
        '''
        if len(votes[id_val]) != num_data: #if we do not have 24 datapoints
            result = reddit.submission(id=id_val) #go to the specific post
            votes[id_val].append(result.ups - result.downs) #get the current number of upvotes and add to the list
            totalcomments[id_val].append(result.num_comments) #get number of comments
        else:
            pass #do nothing if we already collected data from a post for 24hrs
        
    for post in sub:
        pid = str(post.id)
        if pid not in votes: #if new post, add it to the dictionary
            votes[pid] = [post.ups - post.downs] #get the upvotes of the post
            totalcomments[pid] = [post.num_comments] #get number of comments
            timestamps[pid] = [post.created_utc] #store the timestamps of each post. still need to convert later.
        else:#just in case we get some repeating posts
            pass
    
    upvotes = pd.DataFrame(list(votes.values()),index=votes.keys())
    upvotes.to_csv('./votes.csv',index_label='id',mode='w+')

    comments = pd.DataFrame(list(totalcomments.values()),index=totalcomments.keys())
    comments.to_csv('./comments.csv',index_label='id',mode='w+')

    times = pd.DataFrame(list(timestamps.values()),index=timestamps.keys())
    times.to_csv('./post_time.csv',index_label='id',mode='w+')

    on = pd.DataFrame(list(online.values()),index=online.keys())
    on.to_csv('./online_users.csv',index_label='timestamp',mode='w+')
    
    return votes, totalcomments, timestamps, online

In [33]:
#run this cell once for initialization or when you want to restart data collection

reddit = praw.Reddit(client_id='eOqj6Q2BiQQyKQ', client_secret='1hqeAwOvlkY1PYw51ewhbrItd5k', user_agent='Reddit WebScrapping')

sub_title = 'funny' #title of subreddit to scrape
votes = {} #store post ids and votes. Positive votes and NEGATIVE votes (when post.downs is > 0)
totalcomments = {} #store post ids and the corresponding number of comments
timestamps = {} #store timestamp of post creation just so we can tell what day/hour
online = {'timestamp':[],'users_online':[]} #store number of online users and timestamp of subreddit
num_data = 48 #how many datapoints we want per post, right now it is assuming 24hr with 30min intervals.
subred = reddit.subreddit(sub_title)

In [43]:
#run this cell manually every 1hr or 30mins whenever
#using this method, we cant put a cap on when to stop collecting new data so we just have to manually drop data.
votes, totalcomments, timestamps, online = get_data(sub_title, subred, reddit, votes, totalcomments, timestamps, online, num_data)

Below is some test stuff for reading in the csv and dropping NaN values

In [46]:
a = pd.read_csv('test2_votes.csv')
a

,id,0,1,2,3,4
0,dsrx38,2,4,5,5,5.0
1,dsrvp8,4,4,4,4,4.0
2,dsrvn3,5,5,5,5,5.0
3,dsrvff,13,14,14,14,15.0
4,dsrv4m,4,4,4,4,4.0
5,dsruvh,6,7,7,7,7.0
6,dsruqj,6,6,6,6,8.0
7,dsru7k,1,1,1,0,0.0
8,dsrtsg,3,3,3,3,3.0
9,dsrt0s,9,9,9,9,9.0


In [47]:
a.dropna(inplace=True)

In [48]:
a

,id,0,1,2,3,4
0,dsrx38,2,4,5,5,5.0
1,dsrvp8,4,4,4,4,4.0
2,dsrvn3,5,5,5,5,5.0
3,dsrvff,13,14,14,14,15.0
4,dsrv4m,4,4,4,4,4.0
5,dsruvh,6,7,7,7,7.0
6,dsruqj,6,6,6,6,8.0
7,dsru7k,1,1,1,0,0.0
8,dsrtsg,3,3,3,3,3.0
9,dsrt0s,9,9,9,9,9.0


In [50]:
b = pd.read_csv('test2_online_users.csv')
b

,timestamp,0,1,2,3,4,5,6,7,8,9
0,timestamp,1.573099e+09,1.573099e+09,1.573099e+09,1.573099e+09,1.573099e+09,1.573099e+09,1.573099e+09,1.573099e+09,1.573099e+09,1.573099e+09
1,users_online,4.101500e+04,4.101500e+04,4.101500e+04,4.101500e+04,4.101500e+04,4.101500e+04,4.101500e+04,4.101500e+04,4.101500e+04,4.101500e+04


In [1]:
print('hi')

hi
